In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
import sentencepiece as spm
import re
import os
import time

import glob

### Loading Data

In [2]:
path1 = os.getenv('HOME') + '/aiffel/Deep_Sum/total/09' # 자신에 맞는 숫자 집어넣기
filenames1 = glob.glob(path1 + "/*.csv") 


dfs = []
for filename in filenames1:
    dfs.append(pd.read_csv(filename))

data = pd.concat(dfs, ignore_index=True)
print(data.shape)

(608221, 7)


In [3]:
data = data.drop_duplicates(['contents'], ignore_index = True)

In [4]:
def cleaned_text(text):
    a = re.sub(r'\[', '&& [',text)
    a = re.sub(r'\]', '] &&',a)
    a = re.sub(r'&&', '', a )
    
    # (~~)의 양 옆에 띄어쓰기를 추가
    a = re.sub(r'\(', '&& (',a)
    a = re.sub(r'\)', ') &&',a)
    a = re.sub(r'&&', '', a )
    
    # <~~>의 양옆에 띄어쓰기를 추가
    a = re.sub(r'\<', '&& <',a)
    a = re.sub(r'\>', '> &&',a)
    a = re.sub(r'&&', '', a )
    
    # (~~)부분 삭제
    a = re.sub(r'\([^)]*\)', '', a )
    
    # [~~]부분 삭제
    a = re.sub(r'\[[^)]*\]', '', a )
    
    # <~~>부분 삭제
    
    a = re.sub(r'\<[^)]*\>', '', a )
    
    # 해당 패턴에 해당하는 부분 삭제
    pattern = '[\r|\n|\n\n]'
    a = re.sub(pattern = pattern, repl= ' ', string = a)
    
    # 이메일 삭제
    a = re.sub('[a-zA-Z0-9_-]+@\S+.+[a-zA-Z0-9]', '', a)
    #a = re.sub('[a-zA-Z0-9_-]+@[a-z]+.+[a-z]+.+[a-z]', '', a)
    #a = re.sub('[a-zA-Z0-9_-]+@[a-z]+.[a-z]', '', a)
    #a = re.sub('[a-z]+.[a-z]', '', a)
    
    # 날짜 삭제
    # 필요시 만들기
    
    
    # 마침표 뒤에 띄어쓰기
    #a = re.sub('\.', '. ', a) #숫자의 소수점으로 쓰이는 경우도 있기 때문에 토크나이저에게 맡김 
    
    # 두칸 이상의 띄어쓰기 제거
    a = re.sub('  ', ' ', a)
    a = re.sub('  ', ' ', a)
    a = re.sub('  ', ' ', a)
    
    # 양 옆에 공백제거 
    a = a.strip()
    
    return a

In [5]:
data['cleaned_texts'] = data['contents'].apply(cleaned_text)

In [6]:
document = data['cleaned_texts']
summary = data['title']
print(document.head())
print(summary.head())
print(len(document))

0    이해찬 전 더불어민주당 대표 .© News1 박세연 기자 이해찬 전 더불어민주당 대...
1    박재호 더불어민주당 의원. 사진=뉴스1 박재호 더불어민주당 의원. 사진=뉴스1 사회...
2    문재인 대통령이 11일 오전 정부세종청사 코로나19 중앙사고수습본부를 격려 방문을 ...
3    “태풍이 지나가고 아침저녁으로 신선한 바람 불어보는 가을의 문턱입니다…맑은 가을 하...
4    신정원 이재우 기자 = 로버트 에이브럼스 주한미군사령관 겸 한미연합사령관이 북한이 ...
Name: cleaned_texts, dtype: object
0          이해찬 "야당, 秋에 억지부려…의료계, 이번만큼은 엄히 다스려야"
1              페북에 성인물 버젓이…여당 박재호 윤리특위 간사 내정 논란
2              문대통령 "사랑한다는 말씀드린다"에 중수본서 터져나온 박수
3                  추미애, 전국 검찰청 직원에 “檢개혁 완수” 이메일
4    주한美사령관 "北, 코로나 막으려 국경서 사살 명령…도발 징후 없어"(종합)
Name: title, dtype: object
367367


0    기사 섹션 분류 안내 기사의 섹션 정보는 해당 언론사의 분류를 따르고 있습니다. 언...
1                                               동영상 뉴스
2    유신재 박사. 한국해양과학기술원 제공 유신재 박사. 한국해양과학기술원 제공 황현용 ...
3    지난 1일 요하네스버그 OR탐보 국제공항에서 승객들의 체온을 측정하고 있다.  김성...
4    국내 대표 마스크 제조업체인 ㈜ 강진 아이앤씨가 코로나19로 힘든 시기에 미래의 주...
Name: cleaned_texts, dtype: object
0    추미애 “윤석열, 사과부터 했어야”…윤, 내일 국감 발언 ‘주목’
1            독감 백신 접종 뒤 9명 사망..."1명은 질식사"
2                                  22일 알림
3        남아공 코로나19 재급증 위험…케이프타운 신규확진 42%↑
4         ㈜강진 아이앤씨, 임실군에 어린이 마스크 2만5천장 기부
Name: title, dtype: object

### Preprocessing

#### Tokenizing the texts into integer tokens

In [7]:
# #sentencepiece에 적용하기 위한 txt파일 생성

# with open('document.txt', 'w') as f:
#     for sentence in document:
#         f.write('{}\n'.format(sentence))

In [8]:
# #sentencepiece에 적용하기 위한 txt파일 생성

# with open('summary.txt', 'w') as f:
#     for sentence in summary:
#         f.write('{}\n'.format(sentence))

In [9]:
# def generate_tokenizer(txt):
#     templates= '--input={} \
#     --pad_id=0 --pad_piece=<PAD> \
#     --bos_id=1 --bos_piece=<BOS> \
#     --eos_id=2 --eos_piece=<EOS> \
#     --unk_id=3 --unk_piece=<UNK> \
#     --model_prefix={} \
#     --vocab_size={} \
#     --character_coverage={} \
#     --model_type={}'
    
#     train_input_file = txt
#     vocab_size = 400000 # vocab 사이즈
#     prefix = txt.split('.')[0] + '_spm' # 저장될 tokenizer 모델에 붙는 이름
#     pad_id=0 #<pad> token을 0으로 설정
#     bos_id=1 #<BOS> token을 1으로 설정
#     eos_id=2 #<EOS> token을 2으로 설정
#     unk_id=3 #<UNK> token(unknown)을 3으로 설정
#     character_coverage = 1.0 # to reduce character set
#     model_type ='unigram' # Choose from unigram (default), bpe, char, or word

#     cmd = templates.format(train_input_file,
#                            prefix,
#                            vocab_size,
#                            character_coverage,
#                            model_type)
    
#     spm.SentencePieceTrainer.Train(cmd)
#     sp = spm.SentencePieceProcessor()
#     sp.Load(prefix + '.model')
    
#     return sp

In [10]:
# def generate_tokenizer_summary(txt):
#     templates= '--input={} \
#     --pad_id=0 --pad_piece=<PAD> \
#     --bos_id=1 --bos_piece=<BOS> \
#     --eos_id=2 --eos_piece=<EOS> \
#     --unk_id=3 --unk_piece=<UNK> \
#     --model_prefix={} \
#     --vocab_size={} \
#     --character_coverage={} \
#     --model_type={}'
    
#     train_input_file = txt
#     vocab_size = 100000 # vocab 사이즈
#     prefix = txt.split('.')[0] + '_spm' # 저장될 tokenizer 모델에 붙는 이름
#     character_coverage = 1.0 # to reduce character set
#     model_type ='unigram' # Choose from unigram (default), bpe, char, or word

#     cmd = templates.format(train_input_file,
#                            prefix,
#                            vocab_size,
#                            character_coverage,
#                            model_type)
    
#     spm.SentencePieceTrainer.Train(cmd)
#     sp = spm.SentencePieceProcessor()
#     sp.Load(prefix + '.model')
    
#     return sp

In [11]:
# document_tokenizer = generate_tokenizer('document.txt')
# summary_tokenizer = generate_tokenizer_summary('summary.txt')

In [12]:
document_tokenizer = spm.SentencePieceProcessor(model_file='document_spm.model')
summary_tokenizer = spm.SentencePieceProcessor(model_file='summary_spm.model')

In [13]:
summary_tokenizer.SetEncodeExtraOptions('bos:eos')

True

In [14]:
inputs = []
targets = []

for i in range(0, len(document)):
    src_tokens = document_tokenizer.EncodeAsIds(document[i])
    tgt_tokens = summary_tokenizer.EncodeAsIds(summary[i])
    
    inputs.append(src_tokens)
    targets.append(tgt_tokens)    

In [15]:
encoder_vocab_size = 400000
decoder_vocab_size = 100000

# vocab_size
encoder_vocab_size, decoder_vocab_size

(400000, 100000)

#### Obtaining insights on lengths for defining maxlen

In [16]:
# maxlen
# taking values > and round figured to 75th percentile
# at the same time not leaving high variance
encoder_maxlen = 400
decoder_maxlen = 75

#### Padding/Truncating sequences for identical sequence lengths

In [17]:
inputs = tf.keras.preprocessing.sequence.pad_sequences(inputs, maxlen=encoder_maxlen, padding='post', truncating='post')
targets = tf.keras.preprocessing.sequence.pad_sequences(targets, maxlen=decoder_maxlen, padding='post', truncating='post')

### Creating dataset pipeline

In [18]:
inputs = tf.cast(inputs, dtype=tf.int32)
targets = tf.cast(targets, dtype=tf.int32)

In [19]:
BUFFER_SIZE = 20000
BATCH_SIZE = 64

In [20]:
dataset = tf.data.Dataset.from_tensor_slices((inputs, targets)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

### Positional Encoding for adding notion of position among words as unlike RNN this is non-directional

In [21]:
def get_angles(position, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i // 2)) / np.float32(d_model))
    return position * angle_rates

In [22]:
def positional_encoding(position, d_model):
    angle_rads = get_angles(
        np.arange(position)[:, np.newaxis],
        np.arange(d_model)[np.newaxis, :],
        d_model
    )

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)


### Masking

- Padding mask for masking "pad" sequences
- Lookahead mask for masking future words from contributing in prediction of current words in self attention

In [23]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

In [24]:
def create_look_ahead_mask(size):
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask

### Building the Model

#### Scaled Dot Product

In [25]:
def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)

    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)

    output = tf.matmul(attention_weights, v)
    return output, attention_weights

#### Multi-Headed Attention

In [26]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads

        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)

        self.dense = tf.keras.layers.Dense(d_model)
        
    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])
    
    def call(self, v, k, q, mask):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)
        k = self.wk(k)
        v = self.wv(v)

        q = self.split_heads(q, batch_size)
        k = self.split_heads(k, batch_size)
        v = self.split_heads(v, batch_size)

        scaled_attention, attention_weights = scaled_dot_product_attention(
            q, k, v, mask)

        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])

        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))
        output = self.dense(concat_attention)
            
        return output, attention_weights

### Feed Forward Network

In [27]:
def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='relu'),
        tf.keras.layers.Dense(d_model)
    ])

#### Fundamental Unit of Transformer encoder

In [28]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()

        self.mha = MultiHeadAttention(d_model, num_heads)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
    
    def call(self, x, training, mask):
        attn_output, _ = self.mha(x, x, x, mask)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)

        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)

        return out2


#### Fundamental Unit of Transformer decoder

In [29]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(DecoderLayer, self).__init__()

        self.mha1 = MultiHeadAttention(d_model, num_heads)
        self.mha2 = MultiHeadAttention(d_model, num_heads)

        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)
        self.dropout3 = tf.keras.layers.Dropout(rate)
    
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        attn1, attn_weights_block1 = self.mha1(x, x, x, look_ahead_mask)
        attn1 = self.dropout1(attn1, training=training)
        out1 = self.layernorm1(attn1 + x)

        attn2, attn_weights_block2 = self.mha2(enc_output, enc_output, out1, padding_mask)
        attn2 = self.dropout2(attn2, training=training)
        out2 = self.layernorm2(attn2 + out1)

        ffn_output = self.ffn(out2)
        ffn_output = self.dropout3(ffn_output, training=training)
        out3 = self.layernorm3(ffn_output + out2)

        return out3, attn_weights_block1, attn_weights_block2


#### Encoder consisting of multiple EncoderLayer(s)

In [30]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, maximum_position_encoding, rate=0.1):
        super(Encoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(input_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, self.d_model)

        self.enc_layers = [EncoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)
        
    def call(self, x, training, mask):
        seq_len = tf.shape(x)[1]

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)
    
        for i in range(self.num_layers):
            x = self.enc_layers[i](x, training, mask)
    
        return x


#### Decoder consisting of multiple DecoderLayer(s)

In [31]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self, num_layers, d_model, num_heads, dff, target_vocab_size, maximum_position_encoding, rate=0.1):
        super(Decoder, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        self.embedding = tf.keras.layers.Embedding(target_vocab_size, d_model)
        self.pos_encoding = positional_encoding(maximum_position_encoding, d_model)

        self.dec_layers = [DecoderLayer(d_model, num_heads, dff, rate) for _ in range(num_layers)]
        self.dropout = tf.keras.layers.Dropout(rate)
    
    def call(self, x, enc_output, training, look_ahead_mask, padding_mask):
        seq_len = tf.shape(x)[1]
        attention_weights = {}

        x = self.embedding(x)
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
        x += self.pos_encoding[:, :seq_len, :]

        x = self.dropout(x, training=training)

        for i in range(self.num_layers):
            x, block1, block2 = self.dec_layers[i](x, enc_output, training, look_ahead_mask, padding_mask)

            attention_weights['decoder_layer{}_block1'.format(i+1)] = block1
            attention_weights['decoder_layer{}_block2'.format(i+1)] = block2
    
        return x, attention_weights


#### Finally, the Transformer

In [32]:
class Transformer(tf.keras.Model):
    def __init__(self, num_layers, d_model, num_heads, dff, input_vocab_size, target_vocab_size, pe_input, pe_target, rate=0.1):
        super(Transformer, self).__init__()

        self.encoder = Encoder(num_layers, d_model, num_heads, dff, input_vocab_size, pe_input, rate)

        self.decoder = Decoder(num_layers, d_model, num_heads, dff, target_vocab_size, pe_target, rate)

        self.final_layer = tf.keras.layers.Dense(target_vocab_size)
    
    def call(self, inp, tar, training, enc_padding_mask, look_ahead_mask, dec_padding_mask):
        enc_output = self.encoder(inp, training, enc_padding_mask)

        dec_output, attention_weights = self.decoder(tar, enc_output, training, look_ahead_mask, dec_padding_mask)

        final_output = self.final_layer(dec_output)

        return final_output, attention_weights


### Training

In [33]:
# hyper-params
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
EPOCHS = 30

#### Adam optimizer with custom learning rate scheduling

In [34]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()

        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)


#### Defining losses and other metrics 

In [35]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [36]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

In [37]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)


In [38]:
train_loss = tf.keras.metrics.Mean(name='train_loss')

#### Transformer

In [39]:
transformer = Transformer(
    num_layers, 
    d_model, 
    num_heads, 
    dff,
    encoder_vocab_size, 
    decoder_vocab_size, 
    pe_input=encoder_vocab_size, 
    pe_target=decoder_vocab_size,
)

#### Masks

In [40]:
def create_masks(inp, tar):
    enc_padding_mask = create_padding_mask(inp)
    dec_padding_mask = create_padding_mask(inp)

    look_ahead_mask = create_look_ahead_mask(tf.shape(tar)[1])
    dec_target_padding_mask = create_padding_mask(tar)
    combined_mask = tf.maximum(dec_target_padding_mask, look_ahead_mask)
  
    return enc_padding_mask, combined_mask, dec_padding_mask


#### Checkpoints

In [41]:
checkpoint_path = "checkpoints"

ckpt = tf.train.Checkpoint(transformer=transformer, optimizer=optimizer)

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=50)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print ('Latest checkpoint restored!!')

Latest checkpoint restored!!


In [45]:
checkpoint_path = "checkpoints"
latest = tf.train.latest_checkpoint(checkpoint_path)
ckpt.restore(latest)

#### Training steps

In [43]:
@tf.function
def train_step(inp, tar):
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    enc_padding_mask, combined_mask, dec_padding_mask = create_masks(inp, tar_inp)

    with tf.GradientTape() as tape:
        predictions, _ = transformer(
            inp, tar_inp, 
            True, 
            enc_padding_mask, 
            combined_mask, 
            dec_padding_mask
        )
        loss = loss_function(tar_real, predictions)

    gradients = tape.gradient(loss, transformer.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, transformer.trainable_variables))

    train_loss(loss)

In [44]:
for epoch in range(EPOCHS):
    start = time.time()

    train_loss.reset_states()
  
    for (batch, (inp, tar)) in enumerate(dataset):
        train_step(inp, tar)
    
        # 55k samples
        # we display 3 batch results -- 0th, middle and last one (approx)
        # 55k / 64 ~ 858; 858 / 2 = 429
        if batch % 258 == 0:
            print ('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1, batch, train_loss.result()))
      
    #if (epoch + 1) % 5 == 0:
        #ckpt_save_path = ckpt_manager.save()
        #print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    ckpt_save_path = ckpt_manager.save()
    print ('Saving checkpoint for epoch {} at {}'.format(epoch+1, ckpt_save_path))
    print ('Epoch {} Loss {:.4f}'.format(epoch + 1, train_loss.result()))

    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))


Epoch 1 Batch 0 Loss 5.7349
Epoch 1 Batch 258 Loss 5.2625
Epoch 1 Batch 516 Loss 5.1298
Epoch 1 Batch 774 Loss 5.1052
Epoch 1 Batch 1032 Loss 5.0894
Epoch 1 Batch 1290 Loss 5.0813
Epoch 1 Batch 1548 Loss 5.0757
Epoch 1 Batch 1806 Loss 5.0840
Epoch 1 Batch 2064 Loss 5.0904
Epoch 1 Batch 2322 Loss 5.0954
Epoch 1 Batch 2580 Loss 5.0946
Epoch 1 Batch 2838 Loss 5.0943
Epoch 1 Batch 3096 Loss 5.0864
Epoch 1 Batch 3354 Loss 5.0811
Epoch 1 Batch 3612 Loss 5.0716
Epoch 1 Batch 3870 Loss 5.0639
Epoch 1 Batch 4128 Loss 5.0596
Epoch 1 Batch 4386 Loss 5.0564
Epoch 1 Batch 4644 Loss 5.0507
Epoch 1 Batch 4902 Loss 5.0461
Epoch 1 Batch 5160 Loss 5.0417
Epoch 1 Batch 5418 Loss 5.0353
Epoch 1 Batch 5676 Loss 5.0258
Saving checkpoint for epoch 1 at checkpoints/ckpt-3
Epoch 1 Loss 5.0221
Time taken for 1 epoch: 3455.1916058063507 secs

Epoch 2 Batch 0 Loss 5.4946
Epoch 2 Batch 258 Loss 5.0824
Epoch 2 Batch 516 Loss 4.9710
Epoch 2 Batch 774 Loss 4.9400
Epoch 2 Batch 1032 Loss 4.9242
Epoch 2 Batch 1290 Loss

Epoch 11 Batch 0 Loss 5.2428
Epoch 11 Batch 258 Loss 4.6897
Epoch 11 Batch 516 Loss 4.5195
Epoch 11 Batch 774 Loss 4.4577
Epoch 11 Batch 1032 Loss 4.4291
Epoch 11 Batch 1290 Loss 4.4268
Epoch 11 Batch 1548 Loss 4.4271
Epoch 11 Batch 1806 Loss 4.4452
Epoch 11 Batch 2064 Loss 4.4562
Epoch 11 Batch 2322 Loss 4.4679
Epoch 11 Batch 2580 Loss 4.4708
Epoch 11 Batch 2838 Loss 4.4725
Epoch 11 Batch 3096 Loss 4.4685
Epoch 11 Batch 3354 Loss 4.4680
Epoch 11 Batch 3612 Loss 4.4629
Epoch 11 Batch 3870 Loss 4.4620
Epoch 11 Batch 4128 Loss 4.4611
Epoch 11 Batch 4386 Loss 4.4615
Epoch 11 Batch 4644 Loss 4.4593
Epoch 11 Batch 4902 Loss 4.4573
Epoch 11 Batch 5160 Loss 4.4559
Epoch 11 Batch 5418 Loss 4.4524
Epoch 11 Batch 5676 Loss 4.4471
Saving checkpoint for epoch 11 at checkpoints/ckpt-13
Epoch 11 Loss 4.4445
Time taken for 1 epoch: 3454.8984899520874 secs

Epoch 12 Batch 0 Loss 4.8955
Epoch 12 Batch 258 Loss 4.6651
Epoch 12 Batch 516 Loss 4.4906
Epoch 12 Batch 774 Loss 4.4411
Epoch 12 Batch 1032 Loss

Epoch 20 Batch 4386 Loss 4.3098
Epoch 20 Batch 4644 Loss 4.3076
Epoch 20 Batch 4902 Loss 4.3048
Epoch 20 Batch 5160 Loss 4.3036
Epoch 20 Batch 5418 Loss 4.3021
Epoch 20 Batch 5676 Loss 4.2969
Saving checkpoint for epoch 20 at checkpoints/ckpt-22
Epoch 20 Loss 4.2944
Time taken for 1 epoch: 3447.408590078354 secs

Epoch 21 Batch 0 Loss 4.7913
Epoch 21 Batch 258 Loss 4.5332
Epoch 21 Batch 516 Loss 4.3665
Epoch 21 Batch 774 Loss 4.3066
Epoch 21 Batch 1032 Loss 4.2738
Epoch 21 Batch 1290 Loss 4.2703
Epoch 21 Batch 1548 Loss 4.2704
Epoch 21 Batch 1806 Loss 4.2835
Epoch 21 Batch 2064 Loss 4.2948
Epoch 21 Batch 2322 Loss 4.3046
Epoch 21 Batch 2580 Loss 4.3091
Epoch 21 Batch 2838 Loss 4.3102
Epoch 21 Batch 3096 Loss 4.3044
Epoch 21 Batch 3354 Loss 4.3041
Epoch 21 Batch 3612 Loss 4.2990
Epoch 21 Batch 3870 Loss 4.2974
Epoch 21 Batch 4128 Loss 4.2981
Epoch 21 Batch 4386 Loss 4.2997
Epoch 21 Batch 4644 Loss 4.2967
Epoch 21 Batch 4902 Loss 4.2951
Epoch 21 Batch 5160 Loss 4.2929
Epoch 21 Batch 5418

KeyboardInterrupt: 

### Inference

#### Predicting one word at a time at the decoder and appending it to the output; then taking the complete sequence as an input to the decoder and repeating until maxlen or stop keyword appears

In [46]:
def evaluate(input_document):
    input_document = cleaned_text(input_document)
    input_document = document_tokenizer.EncodeAsIds(input_document)
    input_document = tf.keras.preprocessing.sequence.pad_sequences([input_document], maxlen=encoder_maxlen, padding='post', truncating='post')
    encoder_input = tf.expand_dims(input_document[0], 0)

    decoder_input = [1] #[BOS] index
    output = tf.expand_dims(decoder_input, 0)
    
    for i in range(decoder_maxlen):
        enc_padding_mask, combined_mask, dec_padding_mask = create_masks(encoder_input, output)

        predictions, attention_weights = transformer(
            encoder_input, 
            output,
            False,
            enc_padding_mask,
            combined_mask,
            dec_padding_mask
        )

        predictions = predictions[: ,-1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        if predicted_id == 2: #[EOS] index
            return tf.squeeze(output, axis=0), attention_weights

        output = tf.concat([output, predicted_id], axis=-1)

    return tf.squeeze(output, axis=0), attention_weights


In [47]:
def summarize(input_document):
    # not considering attention weights for now, can be used to plot attention heatmaps in the future
    summarized = evaluate(input_document=input_document)[0].numpy()
    summarized = np.expand_dims(summarized, 0)
    print(summarized[0].tolist())
    return summary_tokenizer.DecodeIds(summarized[0].tolist())  # since there is just one translated document

# 100(정치)

In [56]:
summarize('여권 차기 대선주자군에서 더불어민주당 이낙연 대표와 함께 ‘양강’으로 꼽히는 이재명 경기지사가 1일 스가 요시히데 일본 총리가 한국을 방문하는 일은 없을 것이라고 단언했다. 민족의 명절인 추석날에 일본 총리에 대한 실망감을 드러내며 ‘반일 이미지’를 부각한 셈이다. 최근 여론조사에서 이 대표 지지율이 근소한 차로 1위를 되찾으며 상승세를 탄 것과 무관치 않다는 분석이 나온다. 이 지사는 이날 자신의 페이스북에 ‘스가총리가 방한할 일은 없겠습니다’라는 제목의 글을 올려 “명확한 3권분립으로 정치의 사법 개입이 금지된 대한민국은 정치의 사법 판결 개입은 불법이고 상식적으로 있을 수 없는 일이기 때문에 일본의 ‘징용판결에 대한 정치개입’ 요구를 이해할 수도, 수용할 수도 없다”고 지적했다. 이어 “법적으로나 국민감정으로나 수용 불가능한 조건을 내세우는 것을 보니 스가총리가 방한할 일은 없을 것 같다”고 주장했다. ‘한국이 피고인 일본 기업 자산을 매각하지 않는다고 약속해야 스가 총리가 방한할 수 있다’는 한 일본 관리 발언에 대한 반응으로 보이는 이 지사의 이같은 언급은 이 대표와는 대비된다. 이 대표는 스가 총리 취임을 계기로 “일본의 국운이 상승하고 한·일관계가 개선되길 바란다”며 기대감을 밝힌 바 있다. 이 지사는 “일본과 한국은 복잡하고 미묘한 역사적, 국제정치학적, 외교군사적, 경제사회적 문제들을 해결하기 위해 정치외교와 경제사회 분리, 상호존중과 이해라는 큰 원칙을 지켜야 한다”며 “일본이 아무리 부인해도 침략과 잔혹한 인권침해의 역사는 대한민국에게 역사적 진실이자, 현실”이라고 못박았다. 그는 “위안부, 강제노역 문제는 누가 뭐라하든 가해자인 일본이 만든 문제”라며 “진정한 화해를 위한 사과는 피해자가 용서하고 그만하라 할 때까지 진심으로 하는 것이지 ‘옜다, 사과’로 쉽게 끝낼 수 있는 것이 아니다”고 강조했다. 그러면서 “일본과 한국은 복잡하고 미묘한 역사적, 국제정치학적, 외교 군사적, 경제 사회적 문제들을 해결하기 위해 정치 외교와 경제사회 분리, 상호존중과 이해라는 큰 원칙을 지켜야 한다”고 주문했다.')

[1, 19, 335, 228, 4, 10, 51245, 402, 10, 99982, 29, 62, 108, 16050, 4, 55, 89, 1322, 16, 15, 871, 559, 14, 593]


'“文정부, 이체하고 묻은 것”...페북, 4개 기관에 ‘디지털 뉴딜’ 요구'

### 이재명 "징용판결 '정치개입' 요구한 스가, 방한할 일 없을 것"

In [55]:
summarize('문재인 대통령이 세계무역기구(WTO) 사무총장 선거에 출마한 유명희 산업통상자원부 통상교섭본부장을 돕기 위해 적극 나섰다. 추석인 1일 앙겔라 메르켈 독일 총리와 정상통화를 갖고 유 본부장 지지를 당부했다. 통화는 문 대통령 제의로 이날 오후 6시부터 약 20분간 이어졌다. 양국 정상의 직접 소통은 약 2년만이다. 두 정상은 2018년 10월 아셈(ASEM·아시아유럽정상회의) 정상회의를 계기로 벨기에 브뤼셀에서 정상회담을 한 바 있다. 문 대통령은 지난달 24일엔 유 본부장 지지 요청 서한을 독일 측에 전달한 바 있다. 문 대통령은 이날 통화에서 “한국은 자유무역질서 속에서 성장해왔고 다자무역체제의 수호와 발전이 WTO 중심으로 이뤄져야 한다는 확고한 신념을 갖고 있다”고 설명했다. 그러면서 “유 본부장은 이런 신념을 실현할 수 있는 비전과 역량을 갖추고 있고 WTO를 발전시키고 신뢰를 회복시킬 수 있는 최적임자라고 생각한다”고 강조했다. 메르켈 총리는 “한국의 유명희 후보가 능력과 전문성을 갖춘 적임자로 보고 있다”고 화답했다고 청와대는 전했다. 문 대통령은 또 오는 3일 독일이 통일 30주년을 맞는 것에 대해 “한반도 평화와 통일을 희망하는 우리 국민들에게도 많은 영감을 주는 의미있는 날”이라며 축하의 뜻을 전했다. 이어 “코로나19(신종 코로나바이러스 감염증) 상황이 전세계적으로 다시 악화하면서 우려가 클 것”이라면서 “그동안 (메르켈) 총리 리더십 하에 독일이 코로나19 대응에 모범이 돼온 것에 경의를 표한다”고 밝혔다. “앞으로도 인류가 코로나 위기를 극복하는데 선도적인 역할을 해 줄 것으로 기대한다”는 덕담도 곁들였다. 메르켈 총리는 “한국이 통일에 대해 꾸는 꿈을 잘 알고 있다”며 “성대하게 독일통일 30주년 행사를 치르고 싶었으나 코로나19 때문에 그러지 못해 유감”이라고 말했다. 또 “코로나19 확산을 막아온 한국의 대처 방식에 큰 관심이 있다”고 전했다.')

[1, 1513, 4, 183, 34, 8, 54, 25, 6, 282, 1049, 2872, 5, 244, 11, 525, 768, 251]


"EU, 2020년 '코로나19' 대응 전략 수립...美·中 갈등 확산"

### 文 대통령 “유명희 지지 요청”… 메르켈 “전문성 갖춘 적임자”

# 101(경제)

In [58]:
summarize('[파이낸셜뉴스] 손병석 한국철도(코레일) 사장이 1일 오후 경기도 고양에 있는 수도권철도차량정비단을 찾아 KTX 정비상황을 점검했다. 손병석 사장은 KTX 열차 운행 전 유지관리 상황을 살피고 만일의 상황을 대비한 비상대응체계를 점검했다. 손병석 사장은 “추석 대수송기간 KTX 열차 운행이 늘어난 만큼 더욱 꼼꼼한 정비가 필요하다”며 “특히 방역과 차내설비 점검에 만전을 기해 국민들이 안심하고 편안히 고향을 다녀올 수 있도록 최선을 다해달라”고 강조하고 직원들을 격려했다. 한편 한국철도는 4일까지를 추석 특별교통대책기간으로 정하고, 특별교통대책본부를 24시간 운영하고 있다.')

[1, 8, 871, 2574, 6, 132, 297]


"'디지털 교도소' 운영 중단"

### 손병석 한국철도 사장, 고양 KTX차량기지 안전 점검

In [59]:
summarize("[아시아경제 조강욱 기자] 신용이 개선된 대출 고객들이 은행에 금리 인하를 요구해 3년 반 동안 아낀 돈이 1130억원을 넘어선 것으로 나타났다. 1일 국회 정무위원회 간사인 더불어민주당 김병욱 의원이 금융감독원으로부터 제출받은 자료에 따르면 KB국민ㆍ신한ㆍ우리ㆍ하나ㆍSC제일ㆍ씨티ㆍ기업은행과 케이ㆍ카카오뱅크에 올해 상반기 접수된 금리 인하 요구는 33만8082건으로 집계됐다. 2017년 11만371건에서 2018년 22만8558건, 2019년 47만8150건으로 늘어난 데 이어 증가세가 더 가팔라진 추세다. 금리인하요구권은 신용평가 등급이 올랐거나 취업ㆍ승진을 했을 때, 재산이 늘었을 때 개선된 신용 상태를 반영해 대출 이자를 깎아달라고 요구할 수 있는 권리로 지난해 6월 법제화를 계기로 활성화됐다. 특히 전체 금리 인하 요구 중 비대면 신청의 비중은 2017년 60.3%에서 2018년 85.9%, 2019년 95.2%, 2020년 상반기 98.2%로 급증했다. 대부분이 은행 지점 방문 없이 손쉽게 온라인으로 대출 이자를 아끼고 있는 것이다. 금리인하요구권 수용 건수는 2017년 4만5820건에서 2018년 6만877건, 2019년 14만3059건, 올 상반기 14만3059건으로 늘었다. 또 수용에 따른 이자 절감 추정액은 같은 기간 438억800만원, 327억9200만원, 277억3100만원, 93억2200만원 등으로 매년 감소했다. 금리인하요구권 수용률은 2017년 41.5%, 2018년 26.6%, 2019년 29.9%, 올해 상반기 32.5% 수준이었다. 김병욱 의원은 '더 많은 국민이 자신의 권리를 누리고 혜택을 받을 수 있도록 은행권과 금융당국이 적극적으로 금리인하요구권을 홍보하고 수용률 제고를 위해 노력해야 한다'고 강조했다.")

[1, 9, 10340, 165, 6490, 21, 92, 886, 40, 54, 25, 16, 1149, 94, 261, 1725]


'"신생아 투자하려면도 안 돼"...코로나19에 태풍까지 피해 속출'

### 금리인하 비대면 신청 급증…3년간 아낀 이자 '1100억'

# 105(IT)

In [64]:
summarize("(서울=연합뉴스) 이효석 기자 = 마이크로소프트(MS)는 신형 노트북 '서피스 랩탑 고'(Suface Laptop Go)를 출시한다고 1일 발표했다. MS는 '신종 코로나바이러스 감염증(코로나19)으로 인한 재택근무와 비대면 학습 환경이 장기화하고 있다'면서 '뉴노멀 시대 연결성을 지원하기 위해 서피스 제품의 디자인·색상·가격대를 다양화한다'고 밝혔다. 국내 출시 일정과 가격은 정해지지 않았다. 서피스 랩탑 고는 그동안 출시된 서피스 랩탑 중 가장 가볍고 합리적인 가격으로 나온다고 MS는 설명했다. 12.4인치 픽셀 센스 터치스크린 디스플레이가 장착됐고, 대형 트랙패드와 1.3㎜ 키보드가 정확한 타이핑을 돕는다. 16GB RAM과 256GB 스토리지, 인텔 10세대 i5 쿼드코어 프로세서가 탑재됐다. 구성은 성능과 배터리 수명에 최적화됐다. 배터리 사용시간은 최대 13시간이다. 색상은 아이스 블루, 샌드스톤, 플래티넘 등 3가지로 출시된다. 720p HD 카메라와 스튜디오 마이크는 원격 근무 또는 학습을 지원한다. 옴니소닉 스피커와 돌비 오디오는 영상 통화나 영화·음악 감상 경험을 배가한다. 서피스 랩탑 고는 원터치 지문 인식 로그인 기능으로 안전한 로그인도 지원한다. 원터치 로그인 기능은 원드라이브(OneDrive)의 개인 파일 접근에도 연동된다. MS는 '마이크로소프트365와 온라인 저장 공간 등 클라우드 연결 경험을 극대화했다'면서 '서피스 제품을 처음 쓰는 초보자도 빠르게 적용할 수 있을 것'이라고 덧붙였다. 서피스 프로 X [마이크로소프트 제공. 재판매 및 DB 금지] 이날 MS는 지난해 10월 출시한 태블릿 PC '서피스 프로 X'(Surface Pro X)의 업데이트 버전도 출시한다고 발표했다. 서피스 프로 X 업데이트 버전은 차세대 프로세서가 탑재돼 성능이 빨라지는 게 가장 큰 특징이다. MS SQ1 프로세서의 향상된 버전인 SQ2 프로세서가 탑재돼 현존하는 ARM 기반의 PC 중 가장 빠른 성능을 선보인다고 회사 측은 설명했다. 전력 및 성능 향상으로 배터리 사용시간은 최대 15시간으로 길어졌다. 아이 컨택트(Eye Contact) 기능은 화상 통화를 할 때 서로 눈을 맞추면서 통화하는 느낌이 들도록 시선을 조정해줘서 커뮤니케이션을 돕는다. 시그니처 키보드는 플래티넘, 아이스 블루, 파피레드 등 세 가지 색상이 새로 출시된다. 서피스 프로 X 업데이트 버전의 국내 출시 일정과 가격은 조만간 발표된다. MS는 디자이너 콤팩트 키보드, 무선 블루투스 숫자 패드, 4K 무선 디스플레이 어댑터, 블루투스 인체공학 마우스 등 서피스 제품군을 위한 새로운 액세서리도 출시한다고 덧붙였다.")

[1, 10610, 4, 8, 16207, 28727, 30015, 6, 358, 69]


"MS, '로드 오브 히어로즈' 글로벌 출시"

### 마이크로소프트, 신형 노트북 '서피스 랩탑 고' 출시 발표

In [65]:
summarize("'상온노출' 의심 인플루엔자(독감)백신 접종자가 연일 급증하고 있다. 당초 보건당국은 '문제의 백신 물량을 맞은 사람이 없다'고 발표했으나 접종자수가 어제 하루에만 548명이 늘어나 2000명에 육박하고 있는 것이다. 또한 예방접종치짐을 위반한 접종사례도 1479명분에 이른다. 이로인해 보건당국의 발표가 무색할 정도로 접종자가 늘어나면서 예방접종사업 전반에 걸쳐 부실관리에 대한 지적이 나오고 있다. 한편 질병청은 접종자 가운데 '이상 반응'을 신고한 사람이 이날 4명 더 늘어 총 8명이라고 밝혔다. 이상 반응을 새로 신고한 4명 가운데 2명은 오한·두통·메스꺼움 등, 1명은 두드러기, 1명은 설사 증상이 있다고 각각 보고했다. 질병청은 이와 관련해 '접종 이후 증상이 있었으나 호전된 상태'라고 설명했다. 앞서 이상 반응이 있다고 보고된 4명 역시 발열, 오한 등의 증상이 있었으나 호전됐다고 질병청은 전했다. 질병관리청은 1일 '인플루엔자 예방접종사업 관련' 참고자료를 내고 '현재 상온 노출 여부를 조사 중인 정부조달 (백신) 물량을 접종한 건수는 어제(9월 30일) 기준으로 총 1910건(명)'이라고 밝혔다. 질병청이 전날 발표한 1362명에 비해 하루 새 548명 늘어난 것이다. 접종이 이뤄진 날짜별로 보면 9월 21일까지 접종받은 사람이 1261명으로 가장 많았다. 이후 22일 431명, 23일 23명, 24일 22명, 25일 96명, 26일 38명, 27일 18명, 28일 21명 등이다. 질병청이 긴급하게 사용 중단 결정을 내리면서 일선 현장에서 혼선이 빚어질 수 있었다고 판단한 22일 당일을 제외한 전후의 접종 사례 1479명분의 물량은 모두 예방접종 지침을 위반한 것이다. 실제 한 의료기관에서는 돈을 내고 접종을 받은 60명이 정부의 무료 물량으로 무더기로 접종받은 사실이 드러났다. 질병청 관계자는 '(무료 접종) 사업 시작 전(22일 이전)과 중단 고지일 이후(23일 이후)에 접종이 이뤄진 사례는 국가 인플루엔자 예방접종 사업 지침을 미준수한 사례'라고 지적했다. 이어 '사업 중단 당일인 22일에 이뤄진 접종 사례는 사업 중단을 인지하지 못하고 접종한 것으로 보고 있다'며 '각 지방자치단체를 통해 사용 중지된 물량을 사용한 사례를 지속해서 조사하고 있다'고 설명했다. 접종자가 나온 지역은 강원과 울산을 제외한 전국 15개 시도다. 지역별로는 경기 673건, 전북 326건, 인천 214건, 경북 161건, 서울 149건, 부산 109건, 충남 74건, 세종 51건, 대구 46건, 광주 40건, 전남 31건, 대전 17건, 경남 10건, 제주 8건, 충북 1건 등이다. ◇ 전국 231곳 의료기관서 접종…이상반응 신고 4명 늘어 총 8명 '국가 인플루엔자 예방접종 사업 중단'이라는 초유의 사태 속에서 그간 일선 의료현장에서 백신 접종 및 관리를 부실하게 해 온 점도 조사 과정에서 속속 드러나고 있다. 통상 각 의료기관이 자체적으로 구비한 유료 접종 물량과 정부가 제공하는 무료 접종 물량은 별도로 관리해야 하지만 이를 섞어서 관리하거나 돈을 받고 정부 조달 물량을 쓰는 경우도 적지 않은 것으로 보인다. 현재까지 상온 노출이 의심돼 조사 중인 백신으로 접종한 병·의원만 하더라도 전국 231곳에 달한다. 지역별로 보면 경기가 93곳으로 가장 많고 이어 전북 31곳, 대구 22곳, 서울 18곳, 경북 15곳 부산·충남 11곳 등이다. 질병청은 앞서 국가 조달 물량을 공급하는 업체인 '신성약품'이 백신을 배송하는 과정에서 냉장차 문을 열어놓거나 제품을 바닥에 내려놓는 등 '냉장유통'(콜드체인) 원칙을 지키지 않은 사실을 확인하고 지난 21일 밤 사업 중단 방침을 전격 발표했다. 상온 노출이 의심돼 사용이 중단된 백신 물량은 총 578만명분이다. 당초 질병청은 백신 사용 중단을 발표한 직후인 지난달 22일 문제의 백신 접종자가 1명도 없다고 밝혔으나 9월 25일 이후부터 105명→224명→324명→407명→873명→1362명→1910명 등으로 연일 불어나고 있다.")

[1, 13, 107, 12, 9, 54, 25, 28, 2055, 30, 1724, 5, 1050, 24, 32, 17, 76, 44, 7]


'[속보] "코로나19로 인한 소비...수도권서 2명 추가 확진"'

### `상온 노출` 의심 독감백신 접종 전국 231곳서 2000명 육박

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=081&aid=0003141547

In [53]:
summarize(
    '대한감염학회 등 전문가 단체가 국내 코로나19 유행 상황에 대해 거리두기 단계 상향 등 강력한 방역 조치가 없으면 향후 1~2주 안에 하루 1000명에 육박하는 신규 확진자가 발생할 것이라고 경고했다.\
    대한감염학회 등은 20일 성명서를 통해 “현재 코로나19 상황은 더 악화할 가능성이 높다”며 이같이 밝혔다.\
    이들 학회는 “코로나19 바이러스는 낮은 온도, 건조한 환경에서 더 오래 생존하므로 현재 전파 위험이 높아진 상태”라며 “일일 감염재생산 지수가 1.5를 넘어선 상태여서 효과적 조치 없이 1∼2주 경과하면 일일 확진자 수가 1000명에 육박할 것으로 예측된다”고 밝혔다.\
    이들 학회는 “고위험군에 피해가 발생할 위험이 커지고 있고, 코로나19 중환자를 치료할 자원이 빠르게 고갈되고 있다”며 “발병 후 7∼10일쯤 중증으로 악화하는 코로나19 특성을 고려하면 중환자 병상은 1∼2주 내 빠르게 소진될 것”이라고 진단했다. 이에 따라 조기에 선제적으로 강력하게 방역 조치를 해야 한다고 주문했다.')

[1, 20, 25, 4354, 343, 140, 5, 3577, 4, 8, 6779, 151, 6, 53, 315]


"코로나19 중증 환자 증가...국토부, '제로인'으로 결정"

# 감염학회 “거리두기 상향 없으면 하루 확진자 1천명 육박할 것”

"코로나19 확산 속 '코로나19' 확산...코로나19 확산 우려"

'[속보] "코로나19 백신, 방역 강화 위험 없어"'
'코로나19로 확진자 100만명 넘어...국내 발생 60세 이상'

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=025&aid=0003054205

In [49]:
summarize('서울 양천구의 한 인도에서 지나가던 행인을 무차별 폭행한 40대 남성 A씨가 20일 경찰에 붙잡혔다.\
서울 양천경찰서에 따르면, A씨는 지난 18일 오후 9시쯤 지하철 5호선 오목교역 근처에서 길을 가던 최모(42)씨와 어깨를 부딪치자 주먹을 휘두르고 발길질을 하며 수차례 폭행한 혐의를 받고 있다.\
신고를 받은 경찰이 출동했을 땐 A씨는 이미 현장에서 달아난 상태였다. 최씨는 당시 폭행으로 코뼈가 부러지고 뇌출혈 증상이 나타난 것으로 확인됐다.\
경찰은 주변 CCTV 등을 통해 A씨의 신원을 특정해 이틀 만에 덜미를 잡았다. 경찰 관계자는 “당시 A씨가 술에 취했던 것으로 추정된다”며 상해 등 혐의로 A씨를 입건했다.\
곧 A씨를 소환해 구체적인 범행 경위를 파악할 예정”이라고 말했다.')

[1, 223, 4, 8, 871, 2574, 6, 16, 32, 17, 70]


"경찰, '디지털 교도소'에 2명 사망"

# 경찰, 퇴근길 어깨 부딪히자 무차별 폭행한 40대 남성 입건

"[속보] 경찰, '코로나19'에 '묻지마 폭행' 50대 남성 구속영장 기각"

"[속보] '음주운전' 차량 2명 사망...경찰 수사"

"'나는 자연인이다' 심마니, 동료와 함께 숨져"

https://news.naver.com/main/read.nhn?mode=LSD&mid=shm&sid1=102&oid=001&aid=0012030367

In [50]:
summarize("내 신종 코로나바이러스 감염증(코로나19) 확산세가 연일 거세지면서 21일 신규 확진자 수는 300명대 후반을 기록했다.\
전날(363명)보다 다소 늘어나면서 나흘 연속 300명대를 이어갔다.\
이는 수도권 중심의 '2차 유행'이 한창이던 8월 말 수준과 비슷한 상황이다. 당시엔 2차 유행의 정점을 찍었던 8월 27일(441명)을 전후로 4일 연속(320명→441명→371명→323명) 300명 이상이 단 1차례 있었다.\
그만큼 코로나19 상황이 심각하다는 방증으로, 정부도 지난 2∼3월 대구·경북 중심의 '1차 대유행'과 8월 2차 유행에 이어 '3차 유행'이 진행 중이라고 공식 확인한 상태다.\
이 같은 증가세는 기존 감염 사례에서 매일같이 확진자가 나오는 데다 학교나 학원, 종교시설, 각종 소모임 등을 고리로 전국 곳곳에서 크고 작은 집단발병이 연일 새로 발생하는 데 따른 것이다.\
정부는 환자 발생 동향을 주시하면서 수도권 등에 대한 '사회적 거리두기' 2단계 격상까지 열어두고 다각도의 대책을 모색하고 있다.")

[1, 20, 25, 28, 2055, 30, 407, 4, 10, 30595, 211, 53, 1025, 8, 54, 25, 6, 251]


"코로나19로 인한 경기, 방문판매원으로 만든 '코로나19' 확산"

# 신규확진 386명 나흘연속 300명대, 지역 361명…3차 유행 본격화

'[날씨] 전국 요란한 비...수도권 선선한 바람'

'코로나19 신규 확진 69명...지역발생 50명'

'[속보] 코로나19 신규 확진자 69명...사흘째 세자릿수'

'[속보] 신규확진 156명...엿새째 100명대 유지'

In [ ]:
from rouge_score import rouge_scorer

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score(test,b)

In [3]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [10]:
scores = scorer.score(test,b)
scores

{'rouge1': Score(precision=1.0, recall=1.0, fmeasure=1.0),
 'rougeL': Score(precision=1.0, recall=1.0, fmeasure=1.0)}

In [9]:
test = "코로나19로 인한 경기, 방문판매원으로 만든 '코로나19' 확산"
b = "신규확진 386명 나흘연속 300명대, 지역 361명…3차 유행 본격화"